# High-Performance Pandas: eval() and query()

- NumPy and Pandas rely on the creation of temporary intermediate objects. They can cause increased computational time and memory use.
- Pandas v0.13 includes ``eval()`` and ``query()`` functions, which allow you to directly access C-speed operations without the need for intermediate arrays. They rely on the [Numexpr](https://github.com/pydata/numexpr) package.

### How Compound Expressions cause slowdowns
- Consider adding the elements of two arrays in Numpy vs Python:

In [1]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

1.73 ms ± 28.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [2]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

168 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


- Now consider what happens when computing compound expressions. NumPy evaluates each subexpression, which looks like this:

In [3]:
mask = (x > 0.5) & (y < 0.5)
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

- *every intermediate step is explicitly allocated in memory*. If the ``x`` and ``y`` arrays are very large, this is a problem.
- The Numexpr library allows you to eval this compound expression element by element, without the need to allocate full intermediate arrays. The [Numexpr documentation](https://github.com/pydata/numexpr) has more details, but for the time being it is sufficient to say that the library accepts a *string* giving the NumPy-style expression you'd like to compute:

In [4]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')

# NumPy allclose(): returns True if two arrays are element-wise equal within a tolerance.
np.allclose(mask, mask_numexpr)

True

### pandas.eval() for Efficient Operations
- ``eval()`` uses string expressions to compute operations using ``DataFrame``s.

In [9]:
# build 4 dataframes, all 100x100K
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

- How fast using pd.eval() provides ~50% speedup, and uses less memory.

In [10]:
%timeit df1 + df2 + df3 + df4

92.8 ms ± 958 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit pd.eval('df1 + df2 + df3 + df4')

37.3 ms ± 699 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
np.allclose(df1 + df2 + df3 + df4, pd.eval('df1 + df2 + df3 + df4'))

True

### Operations supported by pd.eval()

In [13]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))
df1.head()

,0,1,2
0,180,112,748
1,447,205,487
2,656,100,98
3,90,450,613
4,529,224,530


#### pd.eval() supports all arithmetic operators:

In [14]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

#### ``pd.eval()`` supports all comparison operators, including chained expressions:

In [15]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### ``pd.eval()`` supports the ``&`` and ``|`` bitwise operators:

In [16]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

#### ``pd(eval()`` also supports literal ``and`` and ``or`` in Boolean expressions:

In [17]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### ``pd.eval()`` supports access to object attributes via ``obj.attr``, and indexes via ``obj[index]``:

In [18]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

#### Other operations
- function calls, conditional statements, loops, and more complex constructs are __currently *not* implemented__ in ``pd.eval()``.

### ``DataFrame.eval()`` for Column-Wise Operations

- Just as Pandas uses ``pd.eval()``, ``DataFrame``s have a similar ``eval()`` method. The benefit of the ``eval()`` method is that columns can be referred to *by name*.

In [19]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


- Now we can compute expressions using the column names:

In [20]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

- ``DataFrame.eval()`` allows more succinct evaluation of expressions with the columns. We can treat *column names as variables*.

In [21]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

### Assignment in DataFrame.eval()
- ``DataFrame.eval()``  also allows assignment to any column.

In [22]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [23]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


- Any existing column can be modified:

In [24]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


### Local variables in DataFrame.eval()
- ``DataFrame.eval()`` local Python variables via the '@' syntax:

In [25]:
column_mean = df.mean(1) # local variable

result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

- The ``@`` character marks a *variable name* rather than a *column name*, and lets you efficiently evaluate expressions involving __two "namespaces"__: the namespace of columns, and the namespace of Python objects.

- Note: this ``@`` character is only supported by ``DataFrame.eval()``, not by ``pandas.eval()`` - ``pandas.eval()``  only has access to one (Python) namespace.

## DataFrame.query() Method

Consider the following:

In [26]:
result1 =          df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

- These expressions involve columns of ``DataFrame``s. But it cannot be evaluated using ``DataFrame.eval()``... instead, for this type of filtering operation, you can use the ``query()`` method:

In [27]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

- This is much more efficient than the masking expression above, plus it's much easier to read. Note that ``query()`` also accepts the ``@`` flag to mark local variables:

In [28]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

### Performance: When to Use These Functions

- Memory use is the most predictable aspect. Every compound expression involving NumPy arrays or Pandas ``DataFrame``s will result in implicit creation of temporary arrays:

In [29]:
x = df[(df.A < 0.5) & (df.B < 0.5)]

- Is roughly equivalent to:

In [30]:
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

- If the size of the temporary ``DataFrame``s is significant compared to available system memory (typically several GB) then consider using ``eval()`` or ``query()``. You can check the approximate size of your array in bytes using this:

In [31]:
df.values.nbytes

32000

- ``eval()`` can be faster even when you are not maxing-out your system memory.
- The issue is how your temporary ``DataFrame``s compare to the size of the L1 or L2 CPU cache on your system (typically a few megabytes); if they are much bigger, then ``eval()`` can avoid some potentially slow movement of values between the different memory caches.
- See ["Enhancing Performance" section](http://pandas.pydata.org/pandas-docs/dev/enhancingperf.html) for more detail.